# Optimizing Model Parameters
https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
data_root = "../../data"

In [4]:
training_data = datasets.FashionMNIST(
    root=data_root,
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root=data_root,
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

## Full Implementation

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312659  [   64/60000]
loss: 2.294612  [ 6464/60000]
loss: 2.276668  [12864/60000]
loss: 2.274460  [19264/60000]
loss: 2.253043  [25664/60000]
loss: 2.236363  [32064/60000]
loss: 2.242983  [38464/60000]
loss: 2.210133  [44864/60000]
loss: 2.203595  [51264/60000]
loss: 2.181179  [57664/60000]
Test Error: 
 Accuracy: 40.9%, Avg loss: 2.170460 

Epoch 2
-------------------------------
loss: 2.183749  [   64/60000]
loss: 2.170995  [ 6464/60000]
loss: 2.112157  [12864/60000]
loss: 2.129136  [19264/60000]
loss: 2.087058  [25664/60000]
loss: 2.033120  [32064/60000]
loss: 2.053926  [38464/60000]
loss: 1.977323  [44864/60000]
loss: 1.981255  [51264/60000]
loss: 1.918540  [57664/60000]
Test Error: 
 Accuracy: 52.5%, Avg loss: 1.913260 

Epoch 3
-------------------------------
loss: 1.952492  [   64/60000]
loss: 1.920464  [ 6464/60000]
loss: 1.800521  [12864/60000]
loss: 1.836015  [19264/60000]
loss: 1.742870  [25664/60000]
loss: 1.693023  [32064/600

# Save and Load the Model
https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html

In [9]:
torch.save(model.state_dict(), 'model.pth')